In [3]:
from bokeh.models import  Callback, ColumnDataSource, Rect, Select,CustomJS
from bokeh.plotting import figure, output_notebook, output_file, show, gridplot
from bokeh.models.layouts import VBox,HBox
import numpy as np

In [6]:
#Color Dictionary
redcolor5 = {u'All ages': "#720017", u'70+ years': "#bd0026", u'50-69 years':  "#f03b20", u'15-49 years': "#fd8d3c", u'Under 5 years': "#f4cc63", u'gridline': '#b2ada6', u'background': '#e3e0db', u'axis' : '#aba9a7'}

#Just a sample of my data
country_both = ['China', 'India', 'United States', 'Russia', 'Japan', 'Indonesia', 'Germany', 'United Kingdom', 'Italy', 'Brazil']
ages_gen = ['Under 5 years', '15-49 years', '50-69 years', '70+ years', 'All ages', 'Age-standardized']

height70yr = np.array([919470, 421922, 321125, 193960, 148946, 107822, 97529, 90198, 81107, 76782])
height50to69 = np.array([640496, 626995, 182338, 195472, 40422, 109242, 44161, 33333, 24964, 64429])
height15to49 = np.array([126094, 139420, 26159, 43239, 5480, 39040, 6829, 4163, 3571, 16152])
heightUnder5 = np.array([10210, 43338, 82, 714, 41, 5255, 0, 26, 0, 1201])
zeros = np.zeros(len(country_both))


#Y Values for Stacked bar chart
yUnder5 = heightUnder5 / 2.0
y15to49 = yUnder5 + height15to49/2.0
y50to69 = y15to49 + height50to69/2.0
y70yr = y50to69 + height70yr/2.0


output_file('UW_TobaccoDeath.html')
#output_notebook()

#Figure for Stacked bar chart
p1 = figure(title="Top Countries with Death Due to Tobacco by Age", 
            x_range=country_both, y_range=[0, np.amax([y70yr+height70yr])],
            background_fill=redcolor5['background'], 
            plot_width=700, plot_height = 600,
            outline_line_color= None)


#source for callback
source1 = ColumnDataSource(data=dict(x=country_both, y = yUnder5, y_full = yUnder5, height = heightUnder5, height_full = heightUnder5 ,height_zeros = zeros, y_zeros = heightUnder5 / 2.0))
source2 = ColumnDataSource(data=dict(x=country_both, y = y15to49, y_full = y15to49, height = height15to49, height_full = height15to49,height_zeros = zeros, y_zeros = height15to49 / 2.0))
source3 = ColumnDataSource(data=dict(x=country_both, y = y50to69, y_full = y50to69, height = height50to69, height_full = height50to69,height_zeros = zeros , y_zeros = height50to69 / 2.0))
source4 = ColumnDataSource(data=dict(x=country_both, y = y70yr, y_full = y70yr, height = height70yr, height_full = height70yr,height_zeros = zeros, y_zeros =  height70yr / 2.0))

#Use rect glyphs for stached bars
p1.rect(x ='x', y ='y', width =.8, height = 'height', source = source1, color=redcolor5['Under 5 years'], alpha=0.8, name = "Under 5")
p1.rect(x = 'x', y ='y', width = .8, height ='height', source = source2, color=redcolor5['15-49 years'], alpha=0.8, name = "15 to 49")
p1.rect(x = 'x', y ='y', width = .8, height ='height', source = source3, color=redcolor5['50-69 years'], alpha = .8, name = "50 to 69")
p1.rect(x = 'x', y ='y', width = .8, height ='height', source = source4, color=redcolor5['70+ years'], alpha = .8, name = "70+ yrs")

#Java script Callbacks for age  
#I want this to recognize the 70+ year old drop down selection 
#and change the plot so that the height of the glyph is the same as the y value and the 70 year old glyph is the only one that displays
Callback_Age = CustomJS(args={'source1': source1,'source2': source2,'source3': source3,'source4': source4}, code="""
        var f = cb_obj.get('value');
        var data1 = source1.get('data');
        var data2 = source2.get('data');
        var data3 = source3.get('data');
        var data4 = source4.get('data');
        if (f == 'Under 5 years') {
            data3['height'] = data3['height_zeros'];
            data2['height'] = data2['height_zeros'];
            data4['height'] = data4['height_zeros'];
            data1['y'] = data1['y_zeros'];
            data1['height'] = data1['height_full'];
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }
        if (f == '15-49 years') {
            data1['height'] = data1['height_zeros'];
            data3['height'] = data3['height_zeros'];
            data4['height'] = data4['height_zeros'];
            data2['y'] = data2['y_zeros'];
            data2['height'] = data2['height_full'];
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }

        if (f == '50-69 years') {
            data1['height'] = data1['height_zeros'];
            data2['height'] = data2['height_zeros'];
            data4['height'] = data4['height_zeros'];
            data3['y'] = data3['y_zeros'];
            data3['height'] = data3['height_full'];
            console.log('data3',data3)
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }
        if (f == '70+ years') {
            data1['height'] = data1['height_zeros'];
            data2['height'] = data2['height_zeros'];
            data3['height'] = data3['height_zeros'];
            data4['y'] = data4['y_zeros'];
            data4['height'] = data4['height_full'];
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }
        if (f == 'All ages') {
            data1['height'] = data1['height_full'];
            data1['y'] = data1['y_full'];
            data2['height'] = data2['height_full'];
            data2['y'] = data2['y_full'];
            data3['height'] = data3['height_full'];
            data3['y'] = data3['y_full'];
            data4['height'] = data4['height_full'];
            data4['y'] = data4['y_full'];
            source1.trigger('change');
            source2.trigger('change');
            source3.trigger('change');
            source4.trigger('change');
            }



    """)
#Use the Select widget
dropdown_age = Select(title="Ages:", value=ages_gen[4], options= ages_gen,  callback = Callback_Age)

#Display data
filters = VBox(dropdown_age)
layout =  HBox(filters, gridplot([[p1]]))
show(layout)

/usr/local/lib/python3.5/site-packages/bokeh/models/plots.py:654: UserWarning: 
            Plot property 'background_fill' was deprecated in Bokeh
            0.11.0 and will be removed. Use 'background_fill_color' instead.
            
  """)
